In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 3, Finished, Available, Finished, False)

In [2]:
df = spark.read.format("csv").option("header","true").load("Files/Bronzebronze_production_data/production_data.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Bronzebronze_production_data/production_data.csv".
display(df)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 4, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 29787cdd-a751-4543-bf32-c0a9ebf1a643)

In [3]:
from pyspark.sql.functions import col, trim, upper
from pyspark.sql.types import IntegerType, DoubleType

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 5, Finished, Available, Finished, False)

In [4]:
silver_df = (df
.withColumn("ProdID", trim(upper(col("ProdID"))))
.withColumn("PlantID", trim(upper(col("PlantID"))))
    .withColumn("PlannedQty", col("PlannedQty").cast(IntegerType()))
    .withColumn("ActualQty", col("ActualQty").cast(IntegerType()))
    .withColumn("RejectedQty", col("RejectedQty").cast(IntegerType()))
)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 6, Finished, Available, Finished, False)

In [5]:
display(silver_df)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 7, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, c1ef34fc-049d-4e5d-af52-2fd28d97feca)

In [6]:
silver_df = silver_df.withColumn(
    "GoodQty",
    col("ActualQty") - col("RejectedQty")
)


StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 8, Finished, Available, Finished, False)

In [7]:
from pyspark.sql.functions import when

silver_df = silver_df.withColumn("RejectedFlag",
when(col("RejectedQty") > col("ActualQty"), 1).otherwise(0))


StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 9, Finished, Available, Finished, False)

In [8]:
display(silver_df)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 10, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, c216bb13-b231-4a9e-a72e-613e6382310d)

In [9]:
silver_df = silver_df.withColumn(
    "MaterialusedinKGs",
    col("ActualQty") * 0.8
)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 11, Finished, Available, Finished, False)

In [10]:
silver_df = silver_df.withColumn(
    "ProductionVarianceQty",
    col("PlannedQty") - col("ActualQty")
)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 12, Finished, Available, Finished, False)

In [11]:
silver_df = silver_df.withColumn(
    "StandardMaterialCost",
    col("MaterialusedinKGs") * 180
)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 13, Finished, Available, Finished, False)

In [12]:
silver_df = silver_df.withColumn(
    "OverProductionFlag",
    when(col("ActualQty") > col("PlannedQty") * 1.2,1).otherwise(0)
)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 14, Finished, Available, Finished, False)

In [13]:
display(silver_df)

StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 15, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 9670bf30-0f5a-4a8b-8d91-1ae83cd342d9)

In [15]:
silver_path = "abfss://Fabric_Test@onelake.dfs.fabric.microsoft.com/Test_Lakehouse.Lakehouse/Files/Silver Layer Files"
silver_df.write.mode("overwrite") \
    .parquet(f"{silver_path}/silver_production")


StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 17, Finished, Available, Finished, False)

In [16]:
df_check = spark.read.parquet(f"{silver_path}/silver_production")
display(df_check)


StatementMeta(, ca2589fc-385d-437a-b1ea-8d9692b9acff, 18, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 1d295dcb-88f7-4ea3-b546-581fb6596f0f)